# El modelo Ramsey-Cass-Koopmans de crecimiento económico

**Randall Romero Aguilar, PhD**

Este demo se base en uno similar (escrito en Matlab) del libro de texto  <a href="https://mitpress.mit.edu/books/applied-computational-economics-and-finance">Computational Economics and Finance</a> (2001) de Mario Miranda y Paul Fackler.

Para correr este archivo, es necesario contar con la versión Python de CompEcon. Puede instalarse con `pip` usando:

    !pip install compecon --upgrade

<i>Última actualización: 2021-Oct-12</i>
<hr>

## Acerca del modelo de RCK

Se asume que la población (y el trabajo) crecen a la tasa $\frac{\dot{L}}{L}=\xi$, y la productividad del trabajo a la tasa $\frac{\dot{Z}}{Z}=\phi$.

El problema del planificador social se escribe como

\begin{align*}
V(k) &=\max_c \int_{0}^{\infty} \frac{c^{1-\theta}}{1-\theta} e^{-\nu t} \mathrm{d} t \\
\text{sujeto a } \dot{k} &=f(k) - c - (\phi + \xi + \delta) k
\end{align*}

donde $\nu=\rho-(1- \theta)\phi$. El planificador debe decidir cuánto debe la sociedad consumir e invertir, dado el nivel de producción.

La ecuación de Hamilton-Jacobi-Bellman es

\begin{equation*}
\nu V(k) = \max_{c}\left\{u(c) + V'(k)[f(k) - c - (\phi + \xi + \delta) k]\right\}
\end{equation*}

La condición de primer orden es

\begin{align*}
u'(c) &= V'(k) &(\partial . /\partial t)\Rightarrow\qquad u''(c)\dot{c} &= V''(k)\dot{k}
\end{align*}

La condición de la envolvente es

\begin{align*}
\nu V'(k) &= V'(k)[f'(k)-(\phi + \xi + \delta)] + V''(k)\dot{k}\\
V''(k)\dot{k} &= -V'(k)[f'(k)-(\phi + \xi + \delta) - \nu] 
\end{align*}

En el óptimo, debe cumplirse este sistema de ecuaciones diferenciales:

\begin{align*}
\dot{c} &= \tfrac{1}{\rho}[f'(k)-(\phi + \xi + \delta) - \nu] c\\
\dot{k} &=f(k) - c - (\phi + \xi + \delta) k
\end{align*}



* Variable de estado
  - k     stock de capital
* Variable de control
  - c     tasa de consumo
* Parámetros
  - 𝛼    participación del capital
  - 𝛿    tasa de depreciación del capital
  - 𝜃    aversión relativa al riesgo
  - 𝜌    tasa continua de descuento
  - 𝜙    tasa de crecimiento de la productividad del trabajo
  - 𝜉    tasa de crecimiento de la población


## Tareas preliminares

### Importar los paquetes relevantes

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from compecon import BasisChebyshev, OCmodel, ODE

## Crear una clase para representar el modelo

In [ ]:
class RamseyCassKoopmans:
    def __init__(self, 𝛼,𝛿,𝜃,𝜌,𝜙,𝜉):
        self.𝛼 = 𝛼   # participación del capital
        self.𝛿 = 𝛿   # tasa de depreciación del capital
        self.𝜃 = 𝜃   # aversión relativa al riesgo
        self.𝜌 = 𝜌   # tasa continua de descuento
        self.𝜙 = 𝜙   # tasa de crecimiento de la productividad del trabajo
        self.𝜉 = 𝜉   # tasa de crecimiento de la población
        self.𝜈 = 𝜌-(1-𝜃)*𝜙 # tasa de descuento ajustada por crecimiento de la productividad
        
        self.f = lambda k: k**𝛼              # función de producción
        self.df = lambda k: 𝛼*k**(𝛼-1)       # producto marginal
        self.u = lambda c: c**(1-𝜃) / (1-𝜃)  # función de utilidad
        self.du = lambda c: c**(-𝜃)          # utilidad marginal  
        self.calcular_estado_estacionario()
        
    @property
    def parámetros(self):
        return self.𝛼, self.𝛿, self.𝜃, self.𝜌, self.𝜙, self.𝜉, self.𝜈
    
    def control(self, k, Vk):
        𝜃 = self.𝜃
        return Vk**(-1/𝜃)

    def reward(self, k, c):
        𝜃 = self.𝜃
        return (1/(1-𝜃) * c**(1-𝜃))

    def transition(self, k, c):
        𝛼,𝛿,𝜃,𝜌,𝜙,𝜉,𝜈 = self.parámetros
        return self.f(k) - (𝜙 + 𝜉 + 𝛿)*k - c
    
    def calcular_estado_estacionario(self):
        𝛼,𝛿,𝜃,𝜌,𝜙,𝜉,𝜈 = self.parámetros
        
        kstar = (𝛼/(𝜙+𝜉+𝛿+𝜈))**(1/(1-𝛼))   # capital
        cstar = self.𝑓(kstar) - (𝜙+𝜉+𝛿) * 𝑘star # consumo
        vstar = self.u(cstar)/𝜈                 # valor
        lstar = self.du(cstar)                 # precio sombra
        ystar = self.f(kstar)                   #producción

        self.estado_estacionario = pd.Series(
            [kstar, cstar, vstar, lstar,ystar],
            index=['Capital', 'Consumo', 'Función valor', 'Precio sombra', 'Producción'])
    
    def resolver(self, n, kmin, kmax):
        # Estructura de aproximación
        basis = BasisChebyshev(n, kmin, kmax, labels=['Capital'])  # base de Chebyshev
        
        # Valores iniciales para iteración
        k = basis.nodes
        basis.y = self.u(self.𝜈 * k)
        
        # Resolver el modelo por colocación
        modelo = OCmodel(basis, self.control, self.reward, self.transition, rho=self.𝜈)
        data = modelo.solve()
        data.rename(columns={'control':'Consumo', 'value':'Valor', 'resid':'Residuo'}, inplace=True)
        data['Producción'] = self.f(data['Capital'])
        data['Tasa de ahorro'] = 1 - data['Consumo']/data['Producción']
        data['Precio sombra'] = modelo.Value(data.index, 1)

        self.solución = data
        self.ode = modelo
        
    def simular(self, k0, T):
        data = self.ode.simulate([k0], T)
        data.rename(columns={'$y_0$': 'Capital', 'control':'Consumo'}, inplace=True)
        data.index.name = 'tiempo'
        data['Producción'] = self.f(data['Capital'])
        self.simulación = data
        
        productividad = pd.Series(np.exp(modelo1.𝜙*data.index.values), index=data.index)
        self.simulación_per_cápita = data.multiply(productividad, axis=0)
        
    def dxdt(self, x):
        k, c = x
        𝛼,𝛿,𝜃,𝜌,𝜙,𝜉,𝜈 = self.parámetros
        cdot = (self.df(k) - (𝛿+𝜙+𝜉+𝜈)) * c / 𝜃
        kdot = self.transition(k, c)
        return np.array([kdot, cdot])
    
    def diagrama_fase(self, ax,klim, clim):
        x0 = [self.estado_estacionario['Capital'], self.estado_estacionario['Consumo']]
        
        problem = ODE(self.dxdt,3,x0)
        problem.rk4(n=1000)
        problem.phase(klim, clim,
              title='ODE Phase Diagram',
              ax=ax,
              animated=0
             )
        𝛼,𝛿,𝜃,𝜌,𝜙,𝜉,𝜈 = self.parámetros
        kvals = np.linspace(*klim, 200)
        ax.plot(self.solución['Capital'], self.solución['Consumo'], color='C0', label='Consumo óptimo')
        ax.plot(kvals, self.f(kvals) - (𝜙 + 𝜉 + 𝛿)*kvals, ls='--',  color='C1', label='$\dot{k}=0$')
        ax.axvline(self.estado_estacionario['Capital'], ls='--',  color='C2', label='$\dot{c}=0$')
        ax.set(title='Diagrama de fase', xlabel='Capital', ylabel='Consumo')
        ax.legend(bbox_to_anchor=(0.85, -0.15), ncol=3)

## Resolver el modelo para un caso particular

### Crear una instancia del modelo

In [ ]:
modelo1 = RamseyCassKoopmans(𝛼 = 0.4, 𝛿 = 0.1,𝜃 = 2.0,𝜌 = 0.05,𝜙 = 0.03,𝜉 = 0.02)

### Estado estacionario

In [ ]:
modelo1.estado_estacionario

### Resolver la ecuación Hamilton-Jacobi-Bellman por colocación

In [ ]:
n=21
kmin=1
kmax=7
modelo1.resolver(n, kmin, kmax)
modelo1.solución

## Graficar los resultados

In [ ]:
kstar, cstar,vstar, lstar,ystar = modelo1.estado_estacionario # ['Capital', 'Consumo', 'Función valor', 'Precio sombra', 'Producción']
klim = np.array([kmin, kmax])

def indicar_estado_estacionario(ax, xstar, ystar, xlab, ylab):
    xmin = ax.get_xlim()[0]
    ymin = ax.get_ylim()[0]    
    ax.hlines(ystar, xmin, xstar, colors=['gray'], linestyles=['--'])
    ax.vlines(xstar, ymin, ystar, colors=['gray'], linestyles=['--'])
    ax.annotate(ylab, (xmin, ystar))
    ax.annotate(xlab, (xstar, ymin))
    ax.plot(xstar, ystar, '.', ms=20);

### Política óptima

In [ ]:
fig1, ax = plt.subplots()
modelo1.solución['Consumo'].plot(ax=ax)
ax.set(title='Política óptima de consumo',
       ylabel='Tasa de consumo',
       xlim=klim)
ax.set_ylim(bottom=0)
indicar_estado_estacionario(ax, kstar, cstar, '$k^*$','$c^*$') 

In [ ]:
fig2, ax = plt.subplots()
modelo1.solución.plot(x='Producción', y='Consumo', ax=ax)
ax.set(title='Política óptima de consumo',
       xlabel='Producción',
       ylabel='Tasa de consumo',
       xlim=modelo1.f(klim))
ax.set_ylim(bottom=0)
indicar_estado_estacionario(ax, ystar, cstar, '$y^*$','$c^*$') 

In [ ]:
fig3, ax = plt.subplots()
modelo1.solución.plot(ax=ax, x='Producción', y='Tasa de ahorro', legend=None)
ax.set(title='Tasa óptima de ahorro',
       xlabel='Producción',
       ylabel='Tasa de ahorro',
       xlim=modelo1.f(klim));

### Función Valor

In [ ]:
fig4, ax = plt.subplots()
modelo1.solución['Valor'].plot(ax=ax)
ax.set(title='Función valor',
       ylabel='$V(k)$',
       xlim=klim)
indicar_estado_estacionario(ax, kstar, vstar, '$k^*$','$V^*$') 

### Precio sombra

In [ ]:
fig5, ax = plt.subplots()
modelo1.solución['Precio sombra'].plot(ax=ax)
ax.set(title='Precio sombra',
       ylabel="$V'(k)$",
       xlim=klim)
ax.set_ylim(bottom=0)
indicar_estado_estacionario(ax, kstar, lstar, '$k^*$',"$V'^*$") 

### Residuo

In [ ]:
fig6, ax = plt.subplots()
modelo1.solución['Residuo'].plot(ax=ax)
ax.set(title='Residuo',
       ylabel='Residuo',
       xlim=klim);

## Simular el modelo

In [ ]:
T = 50
modelo1.simular(kmin, T)
modelo1.simulación

### Graficar las sendas simuladas

In [ ]:
fig7, axs = plt.subplots(3, 1, sharex=True)
modelo1.simulación.plot(ax=axs, subplots=True)
ax.set(title='Simulated Capital Stock and Rate of Consumption',
       xlabel='Time',
       ylabel='Quantity',
       xlim=[0, T])

i = 0
for ax, xstar, lab in zip(axs.T, [kstar,cstar,ystar], ['$k^*$', '$c^*$', '$y^*$']):
    ax.axhline(xstar, ls='--', c=f'C{i}')
    ax.annotate(lab, (0, xstar), color=f'C{i}', va='top')
    i+=1

In [ ]:
fig8,ax = plt.subplots()
modelo1.diagrama_fase(ax,klim=[1,7], clim=[0,2.5])

In [ ]:
fig9, ax= plt.subplots()
modelo1.simulación_per_cápita.plot(ax=ax)
ax.set(title='Crecimiento de las variables per cápita',
       ylabel='escala logarítmica',
       yscale='log');

# Dinámica comparativa: Efecto de aumentar la aversión al riesgo

En nuestro escenario base, hemos asumido que el coeficiente de aversión al riesgo del consumidor es $\theta=2$. ¿Cómo se afectaría la dinámica del consumo e ingreso per cápita, partiendo de las misma condición inicial, si los consumidores fuesen más aversos al riesgo, por ejemplo si $\theta=3$?

Para determinarlo, hacemos una nueva copia de la clase `RamseyCassKoopmans` y comparamos la solución de este escenario alternativo con la del escenario base:

In [ ]:
modelo2 = RamseyCassKoopmans(𝛼 = 0.4, 𝛿 = 0.1, 𝜃 = 3.0, 𝜌 = 0.05,𝜙 = 0.03,𝜉 = 0.02)

## Comparando el equilibrio de largo plazo

El cambio en el estado estacionario es

In [ ]:
escenarios = ['Escenario base', 'Mayor aversión al riesgo']

# Juntamos el estado estacionario de los dos modelos en una tabla de pandas
estados_estacionarios = pd.concat([modelo1.estado_estacionario, modelo2.estado_estacionario], keys=escenarios, axis=1)

# Calculamos el cambio porcentual entre los dos escenarios
estados_estacionarios["% cambio"] = 100*(estados_estacionarios['Mayor aversión al riesgo'] / estados_estacionarios['Escenario base'] - 1)
estados_estacionarios

Vemos que al aumentar la aversión al riesgo el consumo de equilibrio de largo plazo cae en un 4.1%, mientras que la producción cae en 7.8% y el capital en 18.5%.

## Comparando la función de política

In [ ]:
modelo2.resolver(n, kmin, kmax)

# Juntamos la solución de los dos modelos en una sola tabla de pandas
soluciones = pd.concat([modelo1.solución, modelo2.solución], keys=escenarios)

# Graficamos el consumo
soluciones['Consumo'].unstack(level=0).plot(title='Consumo');

Vemos que para niveles bajos de capital (y consecuentemente de producción) la mayor aversión al riesgo lleva al consumidor a ahorrar menos, mientras que para niveles altos de capital el consumidor ahorra más que en el estado estacionario. Esto muestra que el consumidor trata de suavizar aún más su nivel de consumo.

## Comparando la dinámica de largo plazo del ingreso y consumos per cápita:

In [ ]:
modelo2.simular(kmin, T)

# Juntamos las simulaciones en una tabla de pandas
simulaciones_per_cápita = pd.concat([modelo1.simulación_per_cápita, modelo2.simulación_per_cápita], keys=escenarios)

# Calculamos el ahorro
simulaciones_per_cápita['Ahorro'] = simulaciones_per_cápita['Producción'] - simulaciones_per_cápita['Consumo']

# Creamos figura con dos ejes, para graficar todo en un solo objeto
fig, axs = plt.subplots(3,1, figsize=[9,9], sharex=True)

# Graficamos el consumo per cápita
simulaciones_per_cápita['Consumo'].unstack(level=0).plot(ax=axs[0], title='Consumo per cápita')


# Graficamos el consumo per cápita
simulaciones_per_cápita['Ahorro'].unstack(level=0).plot(ax=axs[1], title='Ahorro per cápita')

# Graficamos la producción per cápita
simulaciones_per_cápita['Producción'].unstack(level=0).plot(ax=axs[2], title='Ingreso per cápita');

Vemos que en cuando la aversión al riesgo es mayor, en su intento por suavizar más su consumo el consumidor ahorra menos que en el escenario base. Esto provoca que el capital se acumule más lentamente, a su vez causando que el ingreso per cápita crezca más lentamente. En el largo plazo, nótese que al ser el ahorro permanentemente inferior en el escenario de mayor aversión al riesgo, la producción per cápita crece más lentamente y eventualmente el consumo es **menor** que en el escenario base. 

# Ejercicios

Tomando como punto de partida el ejemplo de análisis de dinámica comparativa anterior, determine los efectos sobre el crecimiento económico de los siguientes cambios en la economía. En todos los casos tome como punto de comparación el `modelo1`, el cual denominamos "Escenario base".

## ¿Cómo cambia la dinámica del crecimiento si disminuye la productividad marginal del trabajo?

Cambie $\alpha=0.4$ a $\alpha=0.3$.

## ¿Cómo cambia la dinámica del crecimiento si disminuye el capital físico se deprecia más rápidamente?

Cambie $\delta=0.1$ a $\delta=0.15$.

## ¿Cómo cambia la dinámica del crecimiento si disminuye la impaciencia de los consumidores? Es decir, si descuentan la utilidad futura más lentamente.

Cambie $\rho=0.05$ a $\rho=0.03$.

## ¿Cómo cambia la dinámica del crecimiento si aumenta la tasa de crecimiento de la productividad del trabajo?

Cambie $\phi=0.03$ a $\phi=0.04$.

## ¿Cómo cambia la dinámica del crecimiento si disminuye la tasa de crecimiento de la población?

Cambie $\xi=0.02$ a $\xi=0.01$.